In [4]:
import pandas as pd
from difflib import SequenceMatcher
import re
import pandas as pd
import logging
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium.webdriver.common.action_chains import ActionChains
import os
import re
from urllib.parse import quote
import gc
import csv

## Cleaning data

In [2]:

path_of_data = "./Data/Data_Scrapping_SE2026_Clean + Ditemukan.csv"
df = pd.read_csv(
    path_of_data,
    delimiter=';',
)

df.head()


C:\Users\Administrator\AppData\Local\Temp\ipykernel_9792\1988946464.py:2: DtypeWarning: Columns (0,15,16,17,18,19,20,21,22,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


,idsbr,Query,Actual Place Name,Category,Rating,Address,Phone Number,Website,Latitude,Longitude,...,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12
0,44199339,"TRIMATRA MEGA SATYA, HR Muhammad Ruko Surya In...",PT Trimatra Mega Satya,Kantor Perusahaan,Gagal," Jl. Permata Hr Muhaamad II No.50 Blok D, RT....",Gagal,Gagal,-7.2576579,112.6453145,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44199841,"TIGA ANUGERAH MAKMUR, JL. PULO WONOKROMO NO. 2...",PT Anugerah Tiga Jaya(percetakan Offset Surabaya),Kantor Perusahaan,"4,2","· Jl. Bulak Banteng Kidul No.11, RT.002/RW.04",(031) 3710842,Gagal,-7.220809,112.7607729,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,44199987,"MOMENTUM BERTUMBUH INDONESIA, Jalan Sapudi No. 4",BERTUMBUH,Kantor Perusahaan,Gagal," Jl. Sapudi No.4, Gubeng, Kec. Gubeng, Suraba...",Gagal,Gagal,-7.2735216,112.7425731,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,44200784,"SINERGI SOLUSINDO JAYA, Ruko Galaxy Bumi Perma...",PT Cahaya Sinergi Solusindo,Tidak ada ulasan,Gagal,· Jl. Dukuh Kupang XXV No.28,0815-2865-5556,Gagal,-7.2838807,112.713978,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,44209067,"KARYA ANAK PERSADA, Jl. Bendul Merisi Selatan ...",Jl. Bendul Merisi Selatan XI No.39,Tidak ada ulasan,Gagal,Kec. Wonocolo,Gagal,Gagal,-7.312379,112.7481007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df.drop(df.columns[15:27], axis=1)
df.head()

,idsbr,Query,Actual Place Name,Category,Rating,Address,Phone Number,Website,Latitude,Longitude,Status,Open Status,Operation Hours,Place,Validasi
0,44199339,"TRIMATRA MEGA SATYA, HR Muhammad Ruko Surya In...",PT Trimatra Mega Satya,Kantor Perusahaan,Gagal," Jl. Permata Hr Muhaamad II No.50 Blok D, RT....",Gagal,Gagal,-7.2576579,112.6453145,Aktif,Gagal,Gagal,"TRIMATRA MEGA SATYA, HR Muhammad Ruko Surya In...",Ditemukan
1,44199841,"TIGA ANUGERAH MAKMUR, JL. PULO WONOKROMO NO. 2...",PT Anugerah Tiga Jaya(percetakan Offset Surabaya),Kantor Perusahaan,"4,2","· Jl. Bulak Banteng Kidul No.11, RT.002/RW.04",(031) 3710842,Gagal,-7.220809,112.7607729,Tutup,· Tutup pukul 16.00,Buka · Tutup pukul 16.00 · Buka · Tutup pukul ...,"TIGA ANUGERAH MAKMUR, JL. PULO WONOKROMO NO. 2...",Ditemukan
2,44199987,"MOMENTUM BERTUMBUH INDONESIA, Jalan Sapudi No. 4",BERTUMBUH,Kantor Perusahaan,Gagal," Jl. Sapudi No.4, Gubeng, Kec. Gubeng, Suraba...",Gagal,Gagal,-7.2735216,112.7425731,Aktif,Gagal,Gagal,"MOMENTUM BERTUMBUH INDONESIA, Jalan Sapudi No. 4",Ditemukan
3,44200784,"SINERGI SOLUSINDO JAYA, Ruko Galaxy Bumi Perma...",PT Cahaya Sinergi Solusindo,Tidak ada ulasan,Gagal,· Jl. Dukuh Kupang XXV No.28,0815-2865-5556,Gagal,-7.2838807,112.713978,Aktif,Gagal,Gagal,"SINERGI SOLUSINDO JAYA, Ruko Galaxy Bumi Perma...",Ditemukan
4,44209067,"KARYA ANAK PERSADA, Jl. Bendul Merisi Selatan ...",Jl. Bendul Merisi Selatan XI No.39,Tidak ada ulasan,Gagal,Kec. Wonocolo,Gagal,Gagal,-7.312379,112.7481007,Aktif,Gagal,Gagal,"KARYA ANAK PERSADA, Jl. Bendul Merisi Selatan ...",Ditemukan


In [4]:
df.describe

<bound method NDFrame.describe of            idsbr                                              Query  \
0       44199339  TRIMATRA MEGA SATYA, HR Muhammad Ruko Surya In...   
1       44199841  TIGA ANUGERAH MAKMUR, JL. PULO WONOKROMO NO. 2...   
2       44199987   MOMENTUM BERTUMBUH INDONESIA, Jalan Sapudi No. 4   
3       44200784  SINERGI SOLUSINDO JAYA, Ruko Galaxy Bumi Perma...   
4       44209067  KARYA ANAK PERSADA, Jl. Bendul Merisi Selatan ...   
...          ...                                                ...   
518944   4536115           JAYA SAKTI, JL.DUPAK JAYA IV/42 , JEPARA   
518945   4536218  NUROCH PRANATA ENGINEERING, JL MANYAR SABRANGA...   
518946   4536360  ARA STUDIO, JL. MANYAR JAYA PRAJA 3-D NO. 9 RT...   
518947   4536363  SAPULIDI SENTOSA, JL MANYAR KERTA ADI 19 NO H-...   
518948   4536419  MUTIARA TIMUR, GALAXY MALL LT 4 NO. 407-408A J...   

                                        Actual Place Name  \
0                                  PT Trimatra Mega 

In [5]:
df["Validasi"].unique()


array(['Ditemukan', nan, 'Ditemukan\\', 'Tidak Ditemukan\\',
       '112.7857498', 'Kafetaria', 'Gagal', 'Ditemukan"', '4,6',
       '(031) 8706369', '4,2', '0812-3064-620', '· Closes 4.30\u202fpm',
       'Educational institution', '-7.3361282', 'Universitas Negeri',
       'Tutup Sementara', 'Tidak Ditem"', 'Aktif', '-7.2583527',
       'SARMINI, RUNGKUT LOR II/15', '4.4', 'Tutup Permanen', 'Homestay',
       'Nasichatuddin, Jl Tambah Osowilangun 10/44', '-7.2480202'],
      dtype=object)

In [6]:
# Filter by 'Validasi' column for rows where the value is 'Ditemukan'
df["Validasi"] = (
    df["Validasi"]
    .astype(str)
    .str.strip()
    .str.replace("\\", "", regex=False)
    .str.replace('"', "", regex=False)
)


In [7]:
df["Validasi"].unique()

array(['Ditemukan', 'nan', 'Tidak Ditemukan', '112.7857498', 'Kafetaria',
       'Gagal', '4,6', '(031) 8706369', '4,2', '0812-3064-620',
       '· Closes 4.30\u202fpm', 'Educational institution', '-7.3361282',
       'Universitas Negeri', 'Tutup Sementara', 'Tidak Ditem', 'Aktif',
       '-7.2583527', 'SARMINI, RUNGKUT LOR II/15', '4.4',
       'Tutup Permanen', 'Homestay',
       'Nasichatuddin, Jl Tambah Osowilangun 10/44', '-7.2480202'],
      dtype=object)

In [8]:
df_filtered = df[df["Validasi"] == "Ditemukan"]

In [9]:
print(df_filtered.describe)

<bound method NDFrame.describe of            idsbr                                              Query  \
0       44199339  TRIMATRA MEGA SATYA, HR Muhammad Ruko Surya In...   
1       44199841  TIGA ANUGERAH MAKMUR, JL. PULO WONOKROMO NO. 2...   
2       44199987   MOMENTUM BERTUMBUH INDONESIA, Jalan Sapudi No. 4   
3       44200784  SINERGI SOLUSINDO JAYA, Ruko Galaxy Bumi Perma...   
4       44209067  KARYA ANAK PERSADA, Jl. Bendul Merisi Selatan ...   
...          ...                                                ...   
518944   4536115           JAYA SAKTI, JL.DUPAK JAYA IV/42 , JEPARA   
518945   4536218  NUROCH PRANATA ENGINEERING, JL MANYAR SABRANGA...   
518946   4536360  ARA STUDIO, JL. MANYAR JAYA PRAJA 3-D NO. 9 RT...   
518947   4536363  SAPULIDI SENTOSA, JL MANYAR KERTA ADI 19 NO H-...   
518948   4536419  MUTIARA TIMUR, GALAXY MALL LT 4 NO. 407-408A J...   

                                        Actual Place Name  \
0                                  PT Trimatra Mega 

## Scoring

In [10]:
def clean_text(text):
    """Membersihkan dan normalisasi teks untuk perbandingan"""
    if pd.isna(text):
        return ""
    text = str(text).lower()
    # Hapus karakter khusus tapi pertahankan spasi
    text = re.sub(r'[^\w\s]', ' ', text)
    # Hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [11]:
def calculate_similarity(query, place_name, address):
    """Menghitung similarity score antara query dengan place_name + address"""
    query_clean = clean_text(query)
    place_clean = clean_text(place_name)
    address_clean = clean_text(address)
    
    # Gabungkan place name dan address
    combined = f"{place_clean} {address_clean}"
    
    # Hitung similarity menggunakan SequenceMatcher
    similarity = SequenceMatcher(None, query_clean, combined).ratio()
    
    # Berikan bonus jika place_name sangat mirip dengan query
    place_similarity = SequenceMatcher(None, query_clean, place_clean).ratio()
    
    # Weighted score: 60% dari combined similarity, 40% dari place similarity
    final_score = (similarity * 0.6) + (place_similarity * 0.4)
    
    return final_score


In [12]:
def process_csv(input_file):
    """Memproses CSV dan melakukan deduplikasi berdasarkan kecocokan query"""
    
    print(f"Melakukan cleaning......................................\n")
    df = pd.DataFrame(input_file)
    def clean_text(x):
        if isinstance(x, str):
            return (
                x.replace('', '')
                .replace('\n', ' ')
                .replace('\r', ' ')
                .strip()
            )
        return x
    df_res = df.map(clean_text)
    # print(df)
    
    # Pastikan kolom yang diperlukan ada
    required_cols = ['idsbr', 'Query', 'Actual Place Name', 'Address']
    missing_cols = [col for col in required_cols if col not in df_res.columns]
    if missing_cols:
        print(f"Error: Kolom yang hilang: {missing_cols}")
        return
    
    print(f"Total baris: {len(df)}")
    
    # Tambahkan kolom untuk similarity score dan validasi
    df_res['similarity_score'] = 0.0
    df_res['Validasi'] = 'Tidak Ditemukan'
    
    # Hitung similarity score untuk setiap baris 
    for idx, row in df_res.iterrows():
        score = calculate_similarity(
            row['Query'],
            row['Actual Place Name'],
            row['Address']
        )
        if score > 0.6:
            df_res.at[idx, 'similarity_score'] = score

    # print(df)
    
    # Group by idsbr untuk mencari winner
    grouped = df_res.groupby('idsbr')
    
    processed_rows = []
    
    for idsbr, group in grouped:
        if pd.isna(idsbr) or str(idsbr).strip() == '':
            # Jika idsbr kosong, tandai sebagai tidak ditemukan
            for idx, row in group.iterrows():
                row_dict = row.to_dict()
                row_dict['Validasi'] = 'Tidak Ditemukan'

                processed_rows.append(row_dict)
            continue
        
        # Urutkan berdasarkan similarity score (descending)
        sorted_group = group.sort_values('similarity_score', ascending=False)
        
        # Ambil winner (score tertinggi)
        winner_idx = sorted_group.index[0]
        
        for idx, row in sorted_group.iterrows():
            row_dict = row.to_dict()
            
            if idx == winner_idx:
                # Ini adalah winner
                row_dict['Validasi'] = 'Ditemukan'
                row_dict['idsbr'] = idsbr
            else:
                # Ini adalah loser, kosongkan idsbr
                row_dict['Validasi'] = 'Tidak Ditemukan'
            
            processed_rows.append(row_dict)


    # Buat dataframe baru dari hasil
    result_df = pd.DataFrame(processed_rows)
    # print(result_df)
    
    # Hapus kolom similarity_score (kolom temporary)
    result_df = result_df.drop('similarity_score', axis=1)
    
    # Urutkan ulang kolom agar Validasi di akhir
    cols = [col for col in result_df.columns if col != 'Validasi']
    cols.append('Validasi')
    result_df = result_df[cols]
    
    kolom_utama = [
                'idsbr', 'Query', 'Actual Place Name', 'Category', 'Rating',
                'Address', 'Phone Number', 'Website', 'Latitude', 'Longitude',
                'Status', 'Open Status', 'Operation Hours'
    ]
                # Ambil kolom utama yang benar-benar ada di DataFrame
    kolom_utama_ada = [c for c in kolom_utama if c in result_df.columns]

                # Ambil kolom tambahan (selain kolom utama)
    kolom_tambahan = [c for c in result_df.columns if c not in kolom_utama_ada]

                # Susun ulang kolom
    result_df = result_df[kolom_utama_ada + kolom_tambahan]
    # Simpan ke file output

    # Tampilkan statistik
    return result_df



In [13]:
def scorring_data (df_input:pd.DataFrame):
    def clean_text(x):
        if isinstance(x, str):
            return (
                x.replace('', '')
                .replace('\n', ' ')
                .replace('\r', ' ')
                .strip()
            )
        return x
    df_res = df_input.map(clean_text)
    # print(df)
    
    # Pastikan kolom yang diperlukan ada
    required_cols = ['idsbr', 'Query', 'Actual Place Name', 'Address']
    missing_cols = [col for col in required_cols if col not in df_res.columns]
    if missing_cols:
        print(f"Error: Kolom yang hilang: {missing_cols}")
        return
    
    print(f"Total baris: {len(df)}")

    required_cols = ['idsbr', 'Query', 'Actual Place Name', 'Address']
    missing_cols = [col for col in required_cols if col not in df_res.columns]
    if missing_cols:
        print(f"Error: Kolom yang hilang: {missing_cols}")
        return
    
    # Hitung similarity score untuk setiap baris
    for idx, row in df_res.iterrows():
        score = calculate_similarity(
            row['Query'],
            row['Actual Place Name'],
            row['Address']
        )
        df_res.at[idx, 'similarity_score'] = score
    # df_result = df_res

    return df_res

## Clustering by Country 

In [2]:
# Batas koordinat Surabaya (Update Final 2026)
SURABAYA_BOUNDS = {
    'lat_min': -7.36,  
    'lat_max': -7.15,  
    'lon_min': 112.59, 
    'lon_max': 112.88  
}


In [3]:

def bersihkan_dan_konversi(nilai, tipe='lat'):
    """Membersihkan format titik ganda dan konversi ke float"""
    try:
        if pd.isna(nilai): return None
        
        # Ubah ke string dan bersihkan karakter non-numerik kecuali minus dan titik
        s = re.sub(r'[^0-9\.\-]', '', str(nilai))
        
        # Jika ada lebih dari satu titik, ambil semua angka dan format ulang
        if s.count('.') > 1:
            digits = s.replace('.', '').replace('-', '')
            sign = "-" if "-" in s else ""
            if tipe == 'lat':
                # Asumsi Latitude Surabaya dimulai dengan -7...
                return float(sign + digits[0] + "." + digits[1:])
            else:
                # Asumsi Longitude Surabaya dimulai dengan 112...
                # Menangani kasus '1.127...' atau '112.7...'
                if digits.startswith('112'):
                    return float(digits[:3] + "." + digits[3:])
                else:
                    return float(digits[:3] + "." + digits[3:])
        
        return float(s)
    except:
        return None


In [4]:
def cek_lokasi_surabaya(lat_raw, lon_raw):
    """Mengecek apakah koordinat berada di dalam batas Surabaya"""
    lat = bersihkan_dan_konversi(lat_raw, 'lat')
    lon = bersihkan_dan_konversi(lon_raw, 'lon')
    
    if lat is None or lon is None:
        return "error_koordinat"
    
    # Normalisasi otomatis jika angka meledak (misal -72.5 menjadi -7.25)
    if lat < -10: lat = lat / 10
    if lon > 1000: lon = lon / 10
    
    is_lat_in = SURABAYA_BOUNDS['lat_min'] <= lat <= SURABAYA_BOUNDS['lat_max']
    is_lon_in = SURABAYA_BOUNDS['lon_min'] <= lon <= SURABAYA_BOUNDS['lon_max']
    
    if is_lat_in and is_lon_in:
        return "disurabaya"
    else:
        return "tidak disurabaya"

## Start here

In [16]:
#ambil data yang sudah di bersihkan 
# df = df_filtered

# #scoring process
# df_res = scorring_data(df)
# df_res.head()




Total baris: 509074


,idsbr,Query,Actual Place Name,Category,Rating,Address,Phone Number,Website,Latitude,Longitude,Status,Open Status,Operation Hours,Place,Validasi,similarity_score
0,44199339,"TRIMATRA MEGA SATYA, HR Muhammad Ruko Surya In...",PT Trimatra Mega Satya,Kantor Perusahaan,Gagal," Jl. Permata Hr Muhaamad II No.50 Blok D, RT....",Gagal,Gagal,-7.2576579,112.6453145,Aktif,Gagal,Gagal,"TRIMATRA MEGA SATYA, HR Muhammad Ruko Surya In...",Ditemukan,0.396529
1,44199841,"TIGA ANUGERAH MAKMUR, JL. PULO WONOKROMO NO. 2...",PT Anugerah Tiga Jaya(percetakan Offset Surabaya),Kantor Perusahaan,"4,2","· Jl. Bulak Banteng Kidul No.11, RT.002/RW.04",(031) 3710842,Gagal,-7.220809,112.7607729,Tutup,· Tutup pukul 16.00,Buka · Tutup pukul 16.00 · Buka · Tutup pukul ...,"TIGA ANUGERAH MAKMUR, JL. PULO WONOKROMO NO. 2...",Ditemukan,0.426243
2,44199987,"MOMENTUM BERTUMBUH INDONESIA, Jalan Sapudi No. 4",BERTUMBUH,Kantor Perusahaan,Gagal," Jl. Sapudi No.4, Gubeng, Kec. Gubeng, Suraba...",Gagal,Gagal,-7.2735216,112.7425731,Aktif,Gagal,Gagal,"MOMENTUM BERTUMBUH INDONESIA, Jalan Sapudi No. 4",Ditemukan,0.383541
3,44200784,"SINERGI SOLUSINDO JAYA, Ruko Galaxy Bumi Perma...",PT Cahaya Sinergi Solusindo,Tidak ada ulasan,Gagal,· Jl. Dukuh Kupang XXV No.28,0815-2865-5556,Gagal,-7.2838807,112.713978,Aktif,Gagal,Gagal,"SINERGI SOLUSINDO JAYA, Ruko Galaxy Bumi Perma...",Ditemukan,0.465852
4,44209067,"KARYA ANAK PERSADA, Jl. Bendul Merisi Selatan ...",Jl. Bendul Merisi Selatan XI No.39,Tidak ada ulasan,Gagal,Kec. Wonocolo,Gagal,Gagal,-7.312379,112.7481007,Aktif,Gagal,Gagal,"KARYA ANAK PERSADA, Jl. Bendul Merisi Selatan ...",Ditemukan,0.692079


In [17]:
# df_res['similarity_score'].describe()

count    509074.000000
mean          0.458128
std           0.116164
min           0.026667
25%           0.378615
50%           0.456179
75%           0.536477
max           0.933333
Name: similarity_score, dtype: float64

In [8]:
# === EKSEKUSI DATA ===
file_path = "./Data/df_res_score.csv"

try:
    # Membaca file dengan pemisah ;
    df = pd.read_csv(file_path, delimiter=',', low_memory=False)

    # # 1. Hapus kolom Column1 sampai Column12 jika ada
    # cols_to_drop = [f'Column{i}' for i in range(1, 13) if f'Column{i}' in df.columns]
    # df = df.drop(columns=cols_to_drop)

    # 2. Tambahkan kolom status_lokasi dengan fungsi yang sudah diperbaiki
    df['status_lokasi'] = df.apply(
        lambda row: cek_lokasi_surabaya(row['Latitude'], row['Longitude']), 
        axis=1
    )

    # 3. Simpan hasil
    output_path = "./DATA/clean_final_sby_all_score.csv"
    df.to_csv(output_path, sep=';', index=False)

    print("=== LAPORAN PROSES 2026 ===")
    print(f"Total Data: {len(df)}")
    print(f"Distribusi:\n{df['status_lokasi'].value_counts()}")
    
    # Tes bukti perbaikan untuk data yang Anda tanyakan
    print("\nTes Data Error (contoh -72.527.579):")
    print(f"Hasil: {cek_lokasi_surabaya('-72.527.579', '112.735.948')}")
except Exception as e:
    print(f"Terjadi kesalahan: {e}")

=== LAPORAN PROSES 2026 ===
Total Data: 509074
Distribusi:
status_lokasi
disurabaya          448820
tidak disurabaya     46290
error_koordinat      13964
Name: count, dtype: int64

Tes Data Error (contoh -72.527.579):
Hasil: disurabaya


In [6]:
#grap a data with spesific longtitude 
df = pd.read_csv('./Data/clean_final_sby_all_score.csv',delimiter=';')
display(df)

,idsbr,Query,Actual Place Name,Category,Rating,Address,Phone Number,Website,Latitude,Longitude,Status,Open Status,Operation Hours,Place,Validasi,similarity_score,status_lokasi
0,44199339,"TRIMATRA MEGA SATYA, HR Muhammad Ruko Surya In...",PT Trimatra Mega Satya,Kantor Perusahaan,Gagal," Jl. Permata Hr Muhaamad II No.50 Blok D, RT....",Gagal,Gagal,-7.2576579,112.6453145,Aktif,Gagal,Gagal,"TRIMATRA MEGA SATYA, HR Muhammad Ruko Surya In...",Ditemukan,0.396529,disurabaya
1,44199841,"TIGA ANUGERAH MAKMUR, JL. PULO WONOKROMO NO. 2...",PT Anugerah Tiga Jaya(percetakan Offset Surabaya),Kantor Perusahaan,"4,2","· Jl. Bulak Banteng Kidul No.11, RT.002/RW.04",(031) 3710842,Gagal,-7.220809,112.7607729,Tutup,· Tutup pukul 16.00,Buka · Tutup pukul 16.00 · Buka · Tutup pukul ...,"TIGA ANUGERAH MAKMUR, JL. PULO WONOKROMO NO. 2...",Ditemukan,0.426243,disurabaya
2,44199987,"MOMENTUM BERTUMBUH INDONESIA, Jalan Sapudi No. 4",BERTUMBUH,Kantor Perusahaan,Gagal," Jl. Sapudi No.4, Gubeng, Kec. Gubeng, Suraba...",Gagal,Gagal,-7.2735216,112.7425731,Aktif,Gagal,Gagal,"MOMENTUM BERTUMBUH INDONESIA, Jalan Sapudi No. 4",Ditemukan,0.383541,disurabaya
3,44200784,"SINERGI SOLUSINDO JAYA, Ruko Galaxy Bumi Perma...",PT Cahaya Sinergi Solusindo,Tidak ada ulasan,Gagal,· Jl. Dukuh Kupang XXV No.28,0815-2865-5556,Gagal,-7.2838807,112.713978,Aktif,Gagal,Gagal,"SINERGI SOLUSINDO JAYA, Ruko Galaxy Bumi Perma...",Ditemukan,0.465852,disurabaya
4,44209067,"KARYA ANAK PERSADA, Jl. Bendul Merisi Selatan ...",Jl. Bendul Merisi Selatan XI No.39,Tidak ada ulasan,Gagal,Kec. Wonocolo,Gagal,Gagal,-7.312379,112.7481007,Aktif,Gagal,Gagal,"KARYA ANAK PERSADA, Jl. Bendul Merisi Selatan ...",Ditemukan,0.692079,disurabaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509069,4536115,"JAYA SAKTI, JL.DUPAK JAYA IV/42 , JEPARA",CV.Jaya Sakti,Jasa Pengiriman Kargo,4,· Jl. Gadukan Rukun No.39 A,(031) 3533021,Gagal,-7.232.843,112.723.063,Aktif,Gagal,Gagal,"JAYA SAKTI, JL.DUPAK JAYA IV/42 , JEPARA",Ditemukan,0.480000,disurabaya
509070,4536218,"NUROCH PRANATA ENGINEERING, JL MANYAR SABRANGA...",Department of Engineering Physics,Fakultas,"4,7",Gagal,(031) 5947188,Gagal,-72.836.612,112.795.727,Tutup,· Tutup pukul 17.50,Buka · Tutup pukul 17.50 · Buka · Tutup pukul ...,"NUROCH PRANATA ENGINEERING, JL MANYAR SABRANGA...",Ditemukan,0.411429,disurabaya
509071,4536360,"ARA STUDIO, JL. MANYAR JAYA PRAJA 3-D NO. 9 RT...",ARA Studio,Perusahaan arsitektur,"4,6"," Jalan Manyar Jaya Praja, !V, Kec. Sukolilo, ...",Gagal,Gagal,-72.985.459,1.127.682.619,Aktif,· Tutup pukul 17.00,Gagal,"ARA STUDIO, JL. MANYAR JAYA PRAJA 3-D NO. 9 RT...",Ditemukan,0.447586,disurabaya
509072,4536363,"SAPULIDI SENTOSA, JL MANYAR KERTA ADI 19 NO H-...",sapulidi sentosa,Gagal,5," PQ9H+36J, Jl. Manyar Kerta Adi VII, Manyar S...",Gagal,Gagal,-72.822.812,1.127.754.455,Aktif,Gagal,Gagal,"SAPULIDI SENTOSA, JL MANYAR KERTA ADI 19 NO H-...",Ditemukan,0.485007,disurabaya


In [ ]:
df_new = df[df['status_lokasi'] == 'disurabaya' and df['similarity_score'] > 0.6]
